In [1]:
from urllib.request import urlretrieve
import os

import itk
import numpy as np

from itkwidgets import view
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button

In [2]:
# Download input image
file_name = '901-L.nrrd'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/5ef373379014a6d84edb66f8/download'
    urlretrieve(url, file_name)

In [3]:
image = itk.imread(file_name)

In [52]:
camera = np.array([[-7.028664  , 20.177433  , 45.209637  ],
       [ 4.8549995 ,  9.24      , 48.065998  ],
       [-0.17604062,  0.06524051,  0.9822186 ]], dtype=np.float32)
camera = np.array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,  9.24      , 48.065998  ],
       [-0.22125234,  0.05076638,  0.97389436]], dtype=np.float32)
opacity_gaussians = [[{'position': 0.3,
   'height': 0.7545454545454545,
   'width': 0.2194444444444444,
   'xBias': 0.03777777777777777,
   'yBias': 0}]]
opacity_gaussians = [[{'position': 0.23888888888888893,
   'height': 0.3090909090909091,
   'width': 0.1722222222222221,
   'xBias': 0.04708953460902432,
   'yBias': 0}]]
viewer = view(image,
     cmap='bone_Matlab',
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     ui_collapsed=True,
     camera=camera,
     opacity_gaussians=opacity_gaussians
    )
viewer

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,…

In [12]:
label = Label('Segment bones')
progress = FloatProgress(value=0.0,
                         min=0.0,
                         max=1.0,
                         step=0.001,
                         description='progress:',
                         bar_style='info')
progress
box = HBox([label, progress])
box

In [13]:
bone_segmenter = itk.SegmentBonesInMicroCTFilter.New(image)
cortical_thickness=0.1
bone_segmenter.SetCorticalBoneThickness(cortical_thickness)

def update_progress():
    progress.value = bone_segmenter.GetProgress()
bone_segmenter.AddObserver(itk.ProgressEvent(), update_progress)

bone_segmenter.Update()

progress.bar_style = 'success'
progress.description = 'done.'

In [16]:
label_image = bone_segmenter.GetOutput()
label_file_name = file_name + '-labels.nrrd'
itk.imwrite(label_image, label_file_name)

In [53]:
label_image = bone_segmenter.GetOutput()
view(image=image,
     label_image=label_image,
     opacity_gaussians=opacity_gaussians,
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     camera=camera,
     ui_collapsed=True,
     )

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,…

In [22]:
# Download atlas image
atlas_file_name = '907-L.nrrd'
if not os.path.exists(atlas_file_name):
    url = 'https://data.kitware.com/api/v1/file/5ef375199014a6d84edb6a1f/download'
    urlretrieve(url, atlas_file_name)

In [23]:
atlas = itk.imread(atlas_file_name)

In [26]:
# Download atlas label image
atlas_label_file_name = '907-L-atlas.nrrd'
if not os.path.exists(atlas_label_file_name):
    url = 'https://data.kitware.com/api/v1/file/5ef372559014a6d84edb627e/download'
    urlretrieve(url, atlas_label_file_name)

In [30]:
atlas_label = itk.imread(atlas_label_file_name)
# view(label_image=atlas_label)

In [6]:
center_of_femur_head = np.array([4.072,11.347,43.816]).reshape((1,3))
femur_shaft = np.array([7.713,9.107,42.649]).reshape((1,3))
dent = np.array([3.787,11.172,45.173]).reshape((1,3))

opacity_gaussians = [[{'position': 0.33611111111111114,
   'height': 0.24545454545454548,
   'width': 0.22499999999999998,
   'xBias': 0.51,
   'yBias': 0.4}]]
view(atlas,
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     cmap='bone_Matlab',
     opacity_gaussians=opacity_gaussians,
     point_set_sizes=[15,]*3,
     point_set_representations=['spheres',]*3,
     point_sets=[center_of_femur_head, femur_shaft, dent])

Viewer(background=(0.6, 0.6, 0.6), cmap=['bone_Matlab'], geometries=[], gradient_opacity=1.0, opacity_gaussian…

In [40]:
center_of_femur_head = np.array([4.945,9.225,46.011]).reshape((1,3))
femur_shaft = np.array([7.424,9.474,44.711]).reshape((1,3))
dent = np.array([5.018,9.058,47.535]).reshape((1,3))

opacity_gaussians = [[{'position': 0.33611111111111114,
   'height': 0.24545454545454548,
   'width': 0.22499999999999998,
   'xBias': 0.51,
   'yBias': 0.4}]]
viewer = view(image,
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     cmap='bone_Matlab',
     opacity_gaussians=opacity_gaussians,
     point_set_sizes=[15,]*3,
     point_set_representations=['spheres',]*3,
     point_sets=[center_of_femur_head, femur_shaft, dent])
viewer

Viewer(background=(0.6, 0.6, 0.6), cmap=['bone_Matlab'], geometries=[], gradient_opacity=1.0, opacity_gaussian…

In [46]:
head_center_widgets_top = [ColorPicker(description='Head Center ', concise=True, value='#d60000', disabled=True),
                           Button(description='From last click', tooltip='Set from last clicked slice point', icon='dot-circle-o'),]
head_center_widgets_bottom = [FloatText(value=4.945, description='x:'),
                              FloatText(value=9.225, description='y:'),
                              FloatText(value=46.011, description='z:'),]
def head_center_from_click(change):
    position = viewer.clicked_slice_point.position
    for i in range(3):
        head_center_widgets_bottom[i].value = position[i]
head_center_widgets_top[1].on_click(head_center_from_click)
head_center_box = VBox([HBox(head_center_widgets_top), HBox(head_center_widgets_bottom)])

shaft_widgets_top = [ColorPicker(description='Shaft', concise=True, value='#8c39ff', disabled=True),
                     Button(description='From last click', tooltip='Set from last clicked slice point', icon='dot-circle-o'),]
shaft_widgets_bottom = [FloatText(value=7.424, description='x:'),
                        FloatText(value=9.474, description='y:'),
                        FloatText(value=44.711, description='z:'),]
shaft_box = VBox([HBox(shaft_widgets_top), HBox(shaft_widgets_bottom)])

indent_widgets_top = [ColorPicker(description='Indent', concise=True, value='#018700', disabled=True),
                     Button(description='From last click', tooltip='Set from last clicked slice point', icon='dot-circle-o'),]
indent_widgets_bottom = [FloatText(value=5.018, description='x:'),
                         FloatText(value=9.058, description='y:'),
                         FloatText(value=47.535, description='z:'),]
indent_box = VBox([HBox(indent_widgets_top), HBox(indent_widgets_bottom)])

def update_positions():
    head_center_point = np.array([ft.value for ft in head_center_widgets_bottom]).reshape((1,3))
    shaft_point = np.array([ft.value for ft in shaft_widgets_bottom]).reshape((1,3))
    indent_point = np.array([ft.value for ft in indent_widgets_bottom]).reshape((1,3))
    viewer.point_sets = [head_center_point, shaft_point, indent_point]


def head_center_from_click(change):
    if viewer.clicked_slice_point is None:
        return
    position = viewer.clicked_slice_point.position
    for i in range(3):
        head_center_widgets_bottom[i].value = position[i]
    update_positions()
head_center_widgets_top[1].on_click(head_center_from_click)
def shaft_from_click(change):
    if viewer.clicked_slice_point is None:
        return
    position = viewer.clicked_slice_point.position
    for i in range(3):
        shaft_widgets_bottom[i].value = position[i]
    update_positions()
shaft_widgets_top[1].on_click(shaft_from_click)
def indent_from_click(change):
    if viewer.clicked_slice_point is None:
        return
    position = viewer.clicked_slice_point.position
    for i in range(3):
        indent_widgets_bottom[i].value = position[i]
    update_positions()
shaft_widgets_top[1].on_click(shaft_from_click)
for float_input in head_center_widgets_bottom + shaft_widgets_bottom + indent_widgets_bottom:
    float_input.observe(lambda x: update_positions(), 'value')

position_widgets = VBox([head_center_box, shaft_box, indent_box])
position_widgets

In [47]:
landmark_registration = itk.LandmarkAtlasSegmentationFilter[type(image), type(label_image)].New()
landmark_registration.SetInput(image)
landmark_registration.SetInput(1, atlas)
landmark_registration.SetInputLabels(label_image)
landmark_registration.SetAtlasLabels(atlas_label)

In [48]:
LandmarksType = itk.vector[itk.Point[itk.D, 3]]

input_landmarks = LandmarksType()
input_landmarks.push_back([ft.value for ft in head_center_widgets_bottom])
input_landmarks.push_back([ft.value for ft in shaft_widgets_bottom])
input_landmarks.push_back([ft.value for ft in indent_widgets_bottom])
landmark_registration.SetInputLandmarks(input_landmarks)

atlas_landmarks = LandmarksType()
atlas_landmarks.push_back([4.072,11.347,43.816])
atlas_landmarks.push_back([7.713,9.107,42.649])
atlas_landmarks.push_back([3.787,11.172,45.173])
landmark_registration.SetAtlasLandmarks(atlas_landmarks)

In [49]:
%time landmark_registration.Update()

CPU times: user 45.4 s, sys: 1.41 s, total: 46.8 s
Wall time: 10.7 s


In [51]:
view(image=image, label_image=landmark_registration.GetOutput())

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

In [83]:
input_masked_trabecular = itk.mask_negated_image_filter(image, mask_image=label_image, masking_value=2)
input_masked_cancellous = itk.mask_negated_image_filter(image, mask_image=label_image, masking_value=1)
input_masked = itk.add_image_filter(input_masked_trabecular, input_masked_cancellous)
del input_masked_trabecular
del input_masked_cancellous

In [84]:
atlas_masked = itk.mask_image_filter(atlas, mask_image=atlas_label, masking_value=0)

In [96]:
label_masked_trabecular = itk.mask_negated_image_filter(label_image, mask_image=label_image, masking_value=2)
label_masked_cancellous = itk.mask_negated_image_filter(label_image, mask_image=label_image, masking_value=1)
label_masked = itk.add_image_filter(label_masked_trabecular, label_masked_cancellous)
label_masked = itk.cast_image_filter(label_masked, ttype=(type(label_masked), itk.Image[itk.UC, 3]))
del label_masked_trabecular
del label_masked_cancellous

In [97]:
label_masked_map = itk.label_image_to_label_map_filter(label_masked)
cropped = itk.auto_crop_label_map_filter(label_masked_map, crop_border=[6,]*3)
label_masked_cropped = itk.label_map_to_label_image_filter(cropped)

In [104]:
input_masked_cropped = itk.extract_image_filter(input_masked,
                                                extraction_region=label_masked_cropped.GetLargestPossibleRegion())
# view(input_masked_cropped, label_image=label_masked_cropped)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

In [106]:
transform = landmark_registration.GetFinalTransform()
atlas_affine_transformed = itk.resample_image_filter(atlas_masked,
                                                     use_reference_image=True,
                                                     reference_image=input_masked_cropped,
                                                     transform=transform)

In [108]:
itk.size(atlas_affine_transformed)

itkSize3 ([312, 287, 357])

In [75]:
atlas_label_uchar = itk.cast_image_filter(atlas_label, ttype=(type(atlas_label), itk.Image[itk.UC, 3]))
atlas_map = itk.label_image_to_label_map_filter(atlas_label_uchar)
cropped = itk.auto_crop_label_map_filter(atlas_map, crop_border=[6,]*3)
atlas_label_cropped = itk.label_map_to_label_image_filter(cropped)

In [79]:
print(atlas_label)

Image (0x55a5de0cce40)
  RTTI typeinfo:   itk::Image<short, 3u>
  Reference Count: 2
  Modified Time: 2737
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 2546
  UpdateMTime: 2736
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [585, 416, 814]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [585, 416, 814]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [585, 416, 814]
  Spacing: [0.01, 0.01, 0.01]
  Origin: [1.67, 8.97, 42.128]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.01 0 0
0 0.01 0
0 0 0.01

  PointToIndexMatrix: 
100 0 0
0 100 0
0 0 100

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x55a5de0f3e20)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, short>
      Reference Count: 1
      M

In [78]:
print(atlas_label_cropped)

Image (0x55a5e74f2280)
  RTTI typeinfo:   itk::Image<unsigned char, 3u>
  Reference Count: 1
  Modified Time: 959110
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 959100
  UpdateMTime: 959109
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [106, 0, 0]
    Size: [479, 364, 383]
  BufferedRegion: 
    Dimension: 3
    Index: [106, 0, 0]
    Size: [479, 364, 383]
  RequestedRegion: 
    Dimension: 3
    Index: [106, 0, 0]
    Size: [479, 364, 383]
  Spacing: [0.01, 0.01, 0.01]
  Origin: [1.67, 8.97, 42.128]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.01 0 0
0 0.01 0
0 0 0.01

  PointToIndexMatrix: 
100 0 0
0 100 0
0 0 100

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x55a5d8bbe000)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
    